In [2]:
import numpy as np
import pandas as pd

dataset = pd.DataFrame({
    "PHI_E" : [40, 60, 0,40],
    "LAMBDA_E" : [0, 0, -30,0],
    "PHI_S" : [50, 70, 60,41],
    "LAMBDA_S" : [10, 10, 60,1]
})
dataset

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S
0,40,0,50,10
1,60,0,70,10
2,0,-30,60,60
3,40,0,41,1


In [3]:
e1 = np.array(np.cos(np.radians(dataset["LAMBDA_E"]))*np.cos(np.radians(dataset["PHI_E"])))
e2 = np.array(np.sin(np.radians(dataset["LAMBDA_E"]))*np.cos(np.radians(dataset["PHI_E"])))
e3 = np.array(np.sin(np.radians(dataset["PHI_E"])))

s1 = np.array(np.cos(np.radians(dataset["LAMBDA_S"]))*np.cos(np.radians(dataset["PHI_S"])))
s2 = np.array(np.sin(np.radians(dataset["LAMBDA_S"]))*np.cos(np.radians(dataset["PHI_S"])))
s3 = np.array(np.sin(np.radians(dataset["PHI_S"])))

In [4]:
cos_d = e1*s1 + e2*s2 + e3*s3

sin_d_half = 0.5 * np.sqrt( np.power((s1 - e1),2) + np.power((s2 - e2),2) + np.power((s3 - e3),2))
cos_d_half = 0.5 * np.sqrt( np.power((s1 + e1),2) + np.power((s2 + e2),2) + np.power((s3 + e3),2))

sin_d = 0.5*2*2*sin_d_half * cos_d_half

d = np.degrees(np.arctan(sin_d / cos_d))
d

array([12.22406963, 10.82367924, 90.        ,  1.25625007])

In [5]:
dist_km = d*111.195

print("distance for 1st row = {:.2f} km".format(dist_km[0]))
print("distance for 2nd row = {:.2f} km".format(dist_km[1]))
print("distance for 3rd row = {:.2f} km".format(dist_km[2]))
print("distance for 4th row = {:.2f} km".format(dist_km[3]))

distance for 1st row = 1359.26 km
distance for 2nd row = 1203.54 km
distance for 3rd row = 10007.55 km
distance for 4th row = 139.69 km


## Azimuth calculation

$$ Az = atan2(\cos\phi_s \cos\phi_e \sin(\lambda_s - \lambda_e) , \sin\phi_s - \cos\Delta\sin\phi_e) $$

In [6]:
# Azimuth calculation

y = np.cos(np.radians(dataset["PHI_S"]))*np.cos(np.radians(dataset["PHI_E"]))* np.sin(
    np.radians(dataset["LAMBDA_S"]) - np.radians(dataset["LAMBDA_E"]))
x = np.sin(np.radians(dataset["PHI_S"])) - (cos_d* np.sin(np.radians(dataset["PHI_E"])))
azimuth = np.degrees(np.arctan(y/x))

print("Azimuth for the 1st row = {:.2f}".format(azimuth[0])+chr(176))
print("Azimuth for the 2nd row = {:.2f}".format(azimuth[1])+chr(176))
print("Azimuth for the 3rd row = {:.2f}".format(azimuth[2])+chr(176))
print("Azimuth for the 4th row = {:.2f}".format(azimuth[3])+chr(176))

Azimuth for the 1st row = 31.81°
Azimuth for the 2nd row = 18.44°
Azimuth for the 3rd row = 30.00°
Azimuth for the 4th row = 36.93°


# Developing formula for calculating back azimuth

To calculate backazimuth, the equation can be developed in such a way, just interchanging the coordinates of the given equation and adding 180 for calculating Azimuth.


$$ Az = atan2(\cos\phi_s \cos\phi_e \sin(\lambda_s - \lambda_e) , \sin\phi_s - \cos\Delta\sin\phi_e) $$
$$ Baz = atan2(\cos\phi_e \cos\phi_s \sin(\lambda_e - \lambda_s) , \sin\phi_e - \cos\Delta\sin\phi_s) + 180 $$


In [7]:
#backazimuth calculation

a = np.cos(np.radians(dataset["PHI_E"]))*np.cos(np.radians(dataset["PHI_S"]))*np.sin(
    np.radians(dataset["LAMBDA_E"]) - np.radians(dataset["LAMBDA_S"]))
b = np.sin(np.radians(dataset["PHI_E"])) - (cos_d* np.sin(np.radians(dataset["PHI_S"])))
baz = np.degrees(np.arctan(a/b)) + 180

print("Baz 1st = {:2f}".format(baz[0]) + chr(176))
print("Baz 2nd = {:2f}".format(baz[1]) + chr(176))
print("Baz 3rd = {:2f}".format(baz[2]) + chr(176))
print("Baz 4th = {:2f}".format(baz[3]) + chr(176))

Baz 1st = 218.920943°
Baz 2nd = 207.539077°
Baz 3rd = 270.000000°
Baz 4th = 217.575367°


In [8]:
# generating the entire dataset

dataset["Dist(km)"] = dist_km
dataset["Az(deg)"] = azimuth
dataset["Baz(deg)"] = baz
dataset

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S,Dist(km),Az(deg),Baz(deg)
0,40,0,50,10,1359.255422,31.813918,218.920943
1,60,0,70,10,1203.539013,18.437403,207.539077
2,0,-30,60,60,10007.550000,30.000000,270.000000
3,40,0,41,1,139.688727,36.925885,217.575367


## Comparison with the solutions of Assignment 6

$$ \Delta x = 111.195(km/deg)(\phi_s - \phi_e) $$
$$ \Delta x = 111.195(km/deg)(\lambda_s - \lambda_e)\cos \frac{\phi_e+\phi_s}{2}$$
$$\Delta = \sqrt(\Delta x^2 + \Delta y^2)$$
$$Az = \tan^-{(\frac{\Delta y}{\Delta x})} $$
$$Baz = \tan^-{(\frac{\Delta y}{\Delta x})} + 180 $$

In [9]:
# delX and delY
delX = np.array(111.195*(dataset["PHI_S"] - dataset["PHI_E"]))
delY = np.array(111.195*(dataset["LAMBDA_S"] - dataset["LAMBDA_E"])* 
                np.cos(np.radians((dataset["PHI_E"] + dataset["PHI_S"])/ 2)))

# distance, azimuth, back azimuth to be included as columns of the earthquake dataset
# assigning dist(km), azimuth, and backazimuth using the process, used in assignment 6

dist_km_6 = np.sqrt(np.power(delX, 2) + np.power(delY, 2))
azimuth_6 = np.degrees(np.arctan(delY/delX))
baz_6 = np.degrees(np.arctan(delY/delX)) + 180

In [10]:
# generating an updated dataset for comparison between assignment 6 and 7 

df = dataset.copy()
df["Dist(km)_6"] = dist_km_6
df["Az(deg)_6"] = azimuth_6
df["Baz(deg)_6"] = baz_6
df

,PHI_E,LAMBDA_E,PHI_S,LAMBDA_S,Dist(km),Az(deg),Baz(deg),Dist(km)_6,Az(deg)_6,Baz(deg)_6
0,40,0,50,10,1359.255422,31.813918,218.920943,1361.855060,35.264390,215.264390
1,60,0,70,10,1203.539013,18.437403,207.539077,1207.173294,22.909807,202.909807
2,0,-30,60,60,10007.550000,30.000000,270.000000,10937.315651,52.410911,232.410911
3,40,0,41,1,139.688727,36.925885,217.575367,139.691072,37.249575,217.249575
